# Gathering Data

## Imports

- requests: Necessary to pull data from the internet.
- time: Used to separate our data using time intervals.
- json: Allow us to decode json information obtained from the data source.
- math: Allow us to use $\log$ to determine how long a binary search will take.

In [29]:
import requests
import time
import json
import math

## Pre-Info

Data for this project will be obtained from SendouQ match data, found at `https://sendou.ink/q/match/[ID]`. The ID numbers increase sequentially, with the first match ever having an ID of 1, and the current last match (at the time of writing) having an ID around 45400.

I use a sample match from my most recent one at the time, which can be found with ID [45359](https://sendou.ink/q/match/45359).

The data we're interested in is only two things, the score of every player in the game, and the result of the match. Since this information is shown on the screen, it should be possible to extract this data from the page source.

Looking at the page source, it's not easily decipherable at first glance, especially since I have little knowledge of web-dev. However, after some investigation, there is some json data which contains the data we're looking for which can be found immediately after the phrase `"features/sendouq/routes/q.match.$id":` and taking the next text until the bracket ends.

In [36]:
match_ID = '45359'

def get_match(match_ID):
    # grab the data from the page
    r = requests.get('https://sendou.ink/q/match/' + match_ID)
    start_phrase = '"features/sendouq/routes/q.match.$id":'

    # find the start_phrase in the result
    start_index = r.text.find(start_phrase)

    # copy all text after the start_phrase
    data = r.text[start_index + len(start_phrase):]

    # find the end_index by looking for balanced parentheses
    open_parentheses = 0
    for i, c in enumerate(data):
        if c == '{':
            open_parentheses += 1
        elif c == '}':
            open_parentheses -= 1
        if open_parentheses == 0:
            break
    end_index = i + 1

    # trim off the extra data
    return data[:end_index]

# print out data (use an external tool to visualize the json like https://codebeautify.org/jsonviewer)
print(get_match(match_ID))

{"match":{"id":45643,"alphaGroupId":379223,"bravoGroupId":379231,"createdAt":1714871261,"reportedAt":null,"reportedByUserId":null,"memento":{"modePreferences":{"SZ":[{"userId":27094,"preference":"PREFER"},{"userId":23509,"preference":"PREFER"},{"userId":9800,"preference":"PREFER"},{"userId":41674}],"TC":[{"userId":27094,"preference":"PREFER"},{"userId":23509},{"userId":9800},{"userId":41674}],"RM":[{"userId":27094,"preference":"PREFER"},{"userId":23509,"preference":"AVOID"},{"userId":9800,"preference":"PREFER"},{"userId":41674}],"CB":[{"userId":27094,"preference":"PREFER"},{"userId":23509},{"userId":9800},{"userId":41674}]},"pools":[{"userId":27094,"pool":[{"stages":[2,7,10,15,16,17,21],"mode":"SZ"},{"stages":[2,3,6,8,9,10,19],"mode":"TC"},{"stages":[0,2,6,10,16,17,19],"mode":"RM"},{"stages":[0,2,6,8,10,15,17],"mode":"CB"}]},{"userId":7994,"pool":[{"stages":[0,1,2,5,8,9,10],"mode":"TW"},{"stages":[6,8,10,14,15,16,21],"mode":"SZ"},{"stages":[2,8,9,10,14,16,20],"mode":"TC"},{"stages":[2,

Visualize the data above on any json visualizer, I personally used [jsonbeautifier](https://jsonbeautifier.org/). 

### Finding player ratings

The first data we want to obtain is the ratings of the players. This can be found at `groupAlpha.members[0-3].skill.ordinal` for the left team, and replace `groupAlpha` with `groupBeta` for the right team.

However, this value is not what we might expect. From the website, player ratings, which can be seen by clicking on the pictures of the ranks, show a number around 1200-1700. However, the value in data is a smaller number between 0-50. Therefore, there must be some conversion to obtain the rating from the ordinal.

We can compare ordinals vs. ratings to see how we can write a conversion function. This data is mostly from the game I was using above, and then some data from the match with the ID right before (ID: `45359`).

| ordinal $(o)$ | rating $(\mathrm R)$ |
|---|---|
|23.189129414253898|1348|
|25.8303791446169|1387|
|23.431072851780918|1351|
|19.726415339139816|1296|
|29.049225431933444|1436|
|24.00009753744956|1360|
|38.153938142944824|1572|
|31.82404177844055|1477|
|11.527993868993082|1173|
|7.477124875867283|1112|

If you graph this data, you can find a linear correlation, with the only error being a very small rounding error. Thus, we can create a function to convert from ordinal to rating.
$$\operatorname R(o) = 15o + 1000$$

### Finding the winner of the match

The second data we want to obtain is the winner of the match. There seem to be two ways to do this. The first way, and what I expected the website does to display the winner, is to check `match.mapList[0-6].winnerGroupId`, which gives the ID of the team which won each game of the match. This ID can be checked against `alphaGroupId` or `bravoGroupId` to see which team was the winner. The team that won the most maps is the winner of the match.

However, there is an easier, second way to find the information we want. Since we don't care about the actual score of the game, just the winner, we can check `groupAlpha.members[0].skillDifference.spDiff`. If this value is positive, then the first team was the winner, and if it's negative then the second team won. This is because this value contains the amount of points gained or lost as a result of the match, and since the team that wins will always gain points, and the team that loses will always lose points, this can be used to determine the result of the match.

There are times when the second method will not work. This is when the players do not have ratings before the match started, since there won't be a gain or loss of points when the match ends. However, for this project, we must ignore matches where not all players have a rank, since the ranks of all players is important for our data. Thus, this method should always work for finding the result of the match.

In [18]:
match_ID = '45359'

json_data = json.loads(get_match(match_ID))
alpha_won = json_data['groupAlpha']['members'][0]['skillDifference']['spDiff'] > 0

if alpha_won:
    print("Alpha", end='')
else:
    print("Bravo", end='')
print(" won this game")

Alpha won this game


### Finding match time

We already have all the data we strictly need. However, it can be good to split our data into certain time intervals. For example, SendouQ is split into different season, which are periods of time where the service is active. Between seasons, rating data is partially reset, so at the beginning of seasons might give more noisy and unreliable data. Additionally, some time during season 1 and between October 2nd and October 5th, ratings were added to the match history. Matches before these ones will not have rating data to be pulled.

We want to separate our data and see how different seasons might give different results. Additionally, removing the first two weeks of each seasons may provide useful to get cleaner data.

The creation date of each match is stored at `match.createdAt`. This is stored as seconds since the epoch, which can be converted and compared in python pretty easily.

In [24]:
match_ID = '45359'

json_data = json.loads(get_match(match_ID))
local_time = time.localtime(json_data['match']['createdAt'])

print(time.strftime('%x, %I:%M %p', local_time))

05/02/24, 07:00 PM


## Finding Intervals
*All code cells below are necessary to run for the algorithm at the bottom of the notebook to work.*

Before we start writing algorithms to grab data, we need to consider possibly getting rate limited or ip-banned, which we would like to avoid. Since we're going to be pinging the server many times, it's important to space our requests out so we don't overload the server. To do this, we will define a class which will wrap around the `requests.get` function. This class will regulate our usage based on time, and stop us from potentially spamming the server.

The `SafeRequester` class is initialized with a time interval in *seconds*, which we should wait between requests. Its only member function is `SafeRequester.get(...)`, which takes the same parameters that `requests.get(...)` will.

In [40]:
REQUEST_INTERVAL = 2

class SafeRequester:
    def __init__(self, interval):
        self.last_request = time.monotonic()
        self.interval = interval

    def get(self, *args, **kwargs) -> requests.Response:
        current_time = time.monotonic()
        time_diff = current_time - self.last_request

        # if we haven't waited all of the time interval yet, sleep the remaining
        # amount of time before requesting
        while time_diff < self.interval:
            time.sleep(self.interval - time_diff) 
            current_time = time.monotonic()
            time_diff = current_time - self.last_request
        
        self.last_request = current_time
        return requests.get(*args, **kwargs)

requester = SafeRequester(REQUEST_INTERVAL)

Before we do anything else, let's redefine our `get_match` function from earlier to use this new class. Additionally, split the function into two for when we want to get a request and not the data associated with it.

In [52]:
# grab the request data from the input match ID
def get_match(match_ID: int) -> requests.Response:
    # grab the info from the page
    return requester.get(f'https://sendou.ink/q/match/{match_ID}')

# grab the json data from the given request
def match_json(r: requests.Response) -> list | dict:
    assert(r.ok)

    start_phrase = '"features/sendouq/routes/q.match.$id":'

    # find the start_phrase in the result
    start_index = r.text.find(start_phrase)

    # copy all text after the start_phrase
    data = r.text[start_index + len(start_phrase):]

    # find the end_index by looking for balanced parentheses
    open_parentheses = 0
    in_quotes = False       # flag if we're in quotes
    escaped = False         # flag if we're escaped
    for i, c in enumerate(data):
        # if we're escaped, ignore this character
        if escaped:
            escaped = False
            continue
        if c == '\\':
            escaped = True
            continue

        # When we see a quote, toggle the flag in_quotes. This will prevent us
        # from counting brackets found in names of strings. To see where this
        # would be a problem, see match 37500 where a player has the name
        # "プ{Doωm'"
        if c == '"':
            in_quotes = not in_quotes
            continue
        if in_quotes:
            continue

        if c == '{':
            open_parentheses += 1
        elif c == '}':
            open_parentheses -= 1
        if open_parentheses == 0:
            break
    end_index = i + 1

    # trim off the extra data
    return json.loads(data[:end_index])

# small test to make sure a match with a player named "プ{Doωm'" will work
# match_ID = 37500
# r = get_match(match_ID)
# assert(r.ok)
# j = match_json(r)
# assert(j['match']['isLocked'])

The first match is match `1` (this service is surprisingly not zero-indexed). However, the last match, the most recent one, is always changing. We can find this match using a binary search algorithm. All matches which exist should return a status code of `200`, meaning that it exists. Additionally, all match IDs which don't yet exist should return error code `404`. Since we have a range of values which all appear before the index we want to find, and a different range which appears only after the index, we can use binary search.

In [58]:
OK = 200
NOT_FOUND = 404

# starting value for the last match.
# it's very possible far in the future that this will no longer be greater than 
# the actual last match, which is why this algorithm uses two loops and not one
if 'search_interval' not in locals():
    search_interval = 100000


if 'last_match' in locals():
    L = last_match  # type: ignore
else:
    L = 1

R = search_interval
found_last_match = False

while not found_last_match:
    # define variables for our percentage estimate
    search_range = R - L + 1
    max_searches = math.floor(math.log2(search_range)) + 1
    searches = 0
    print(f"Searching range {L}-{R}:\n0%")

    # search loop using binary search
    while L <= R:
        m = (L + R) // 2
        r = get_match(m)
        if r.ok:
            # Check if this match is locked or not (locked meaning the match
            # has concluded). If the match is unfinished, consider it to the 
            # right of the last match
            if match_json(r)['match']['isLocked']:
                L = m + 1
            else:
                R = m - 1
        elif r.status_code == NOT_FOUND:
            R = m - 1
        else:
            raise RuntimeError(f"Got an unexpected error code {r.status_code} ({r.reason})")
    
        searches += 1
        print(f"{searches/max_searches:.1%}")

    if R < search_interval:
        found_last_match = True
    else:
        # in the rare case that this code is being run years into the future 
        # where there have been more than 100,000 matches played, expanded our
        # search interval and search again
        print(f"There have been more games played than {search_interval}!")
        L = search_interval + 1
        search_interval *= 2
        R = search_interval
        print(f"Searching again up to {search_interval} games.")


# once the loop is complete, R is the last match
last_match = R
print(f"Found the last match: {last_match}.")

Searching range 1-100000:
0%
5.9%
11.8%
17.6%
23.5%
29.4%
35.3%
41.2%
47.1%
52.9%
58.8%
64.7%
70.6%
76.5%
82.4%
88.2%
94.1%
Found the last match: 45656.
